In [ ]:
    from load_feat_pd import *
    from scipy import stats

    import sys
    import json

    np.set_printoptions(precision=2)

    config_filepath = 'config_test.json'

    with open(config_filepath, "r") as json_file:
        config_data = json.load(json_file)

    feats2level = config_data["feats2level"]
    allfeats_batch1 = config_data["allfeats_batch1"]
    allfeats_batch2 = config_data["allfeats_batch2"]
    features_to_load = config_data["features_to_load"]
    merged_data_pkl = config_data["merged_data_pkl"]
    base_folder = config_data["base_folder"]
    list_not_load = config_data["list_not_load"]
    # for feat in allfeats:
    # for feat in allfeats:
    
    acc_df = pd.read_csv('/data/storage1t/exps/SSL4PR/checkpoints/res_avg5/Multimodal_WS_tt_balanced_merged_random_avg5.csv')
    acc_df.rename(columns={'fold_or_id': 'subject_id'}, inplace=True)
    acc_df.drop(columns=['accuracy_1', 'accuracy_2', 'accuracy_3','accuracy_4','accuracy_5'], inplace=True)
    #metadata = load_feat(base_folder_path, feature_name=featname)
    # load_feat(base_folder_path, feature_name='contrast', threeD=True)
    for featname in features_to_load:
        metadata = load_feat_single(base_folder, feature_name=featname)
        df = pd.DataFrame(metadata)
        if feats2level[featname] == 'utt':
            # Compute mean and standard deviation of 'value' for each subject_id
            stats_df = df.groupby("subject_id")["value"].agg(value_mean="mean", value_std="std").reset_index()

            # Identify columns to keep (all except 'value', 'filename', and 'item')
            other_cols = [col for col in df.columns if col not in ["value", "filename", "item"]]

            # Get one representative row per subject (assuming these columns are identical within each subject)
            other_df = df.drop_duplicates("subject_id")[other_cols]

            # Merge the aggregated stats with the other columns based on subject_id
            final_df = pd.merge(other_df, stats_df, on="subject_id", how="left")

            # result_df now contains the group-level statistics and the retained columns
            acc_df["subject_id"] = acc_df["subject_id"].astype(str)
            final_df["subject_id"] = final_df["subject_id"].astype(str)

            # Now perform the inner join using subject_id
            joined_df = pd.merge(acc_df, final_df, on="subject_id", how="inner")
            # for col in ["value_mean", "value_std"]:
            #     print(f"Correlation between {col} of {featname} and accuracy: {joined_df['accuracy_avg'].corr(joined_df[col])}")

            # Calculate top10 and bottom10 subjects based on accuracy_avg.
            top10 = joined_df.nlargest(10, 'accuracy_avg')
            bottom10 = joined_df.nsmallest(10, 'accuracy_avg')
            
            # For each of the selected columns, compute and compare the means of top10 and bottom10.
            for col in ["value_mean", "value_std"]:
                joined_df[col] = pd.to_numeric(joined_df[col], errors='coerce')

                top10_mean = top10[col].mean()
                bottom10_mean = bottom10[col].mean()
                print(f"For feature {featname}, {col}: top10 mean = {top10_mean:.3f}, bottom10 mean = {bottom10_mean:.3f}")

            top20 = joined_df.nlargest(20, 'accuracy_avg')
            bottom20 = joined_df.nsmallest(20, 'accuracy_avg')

            # For each selected column, perform Mann-Whitney U test
            for col in ["value_mean", "value_std"]:
                # Drop NA values before testing
                top_data = top20[col].dropna()
                bottom_data = bottom20[col].dropna()
                U, p_val = stats.mannwhitneyu(top_data, bottom_data, alternative='two-sided')
                # print(f"For feature {featname}, {col}: Mann-Whitney U = {U:.3f}, p-value = {p_val:.3f}")
        elif feats2level[featname] == 'frame':
            # For each row in df, compute the mean and std of the 'value' array,
# and add them as new columns "value_mean" and "value_std"
            df["value_mean"] = df["value"].apply(lambda x: np.mean(x) if isinstance(x, (list, np.ndarray)) else np.nan)
            df["value_std"] = df["value"].apply(lambda x: np.std(x) if isinstance(x, (list, np.ndarray)) else np.nan)
            agg_stats_df = df.groupby("subject_id").agg(
                value_mean_mean=("value_mean", "mean"),
                value_mean_std=("value_mean", "std"),
                value_std_mean=("value_std", "mean"),
                value_std_std=("value_std", "std")
            ).reset_index()

            # Next, get one representative row per subject for all columns except ["value", "filename", "item", "value_mean", "value_std"].
            other_cols = [col for col in df.columns if col not in ["value", "experiment", "filename", "item", "value_mean", "value_std"]]
            other_df = df.drop_duplicates("subject_id")[other_cols]

            # Merge the aggregated statistics with the other retained columns based on subject_id.
            final_df = pd.merge(other_df, agg_stats_df, on="subject_id", how="left")
            # Print the final DataFrame
            acc_df["subject_id"] = acc_df["subject_id"].astype(str)
            final_df["subject_id"] = final_df["subject_id"].astype(str)

            # Now perform the inner join using subject_id
            joined_df = pd.merge(acc_df, final_df, on="subject_id", how="inner")
                            
            # for col in ["value_mean_mean", "value_mean_std", "value_std_mean", "value_std_std"]:
            #     print(f"Correlation between {col} of {featname} and accuracy: {joined_df['accuracy_avg'].corr(joined_df[col])}")
            # # Calculate top10 and bottom10 subjects based on accuracy_avg.
            top10 = joined_df.nlargest(10, 'accuracy_avg')
            bottom10 = joined_df.nsmallest(10, 'accuracy_avg')
            # For each of the selected columns, compute and compare the means of top10 and bottom10.
            for col in ["value_mean_mean", "value_mean_std", "value_std_mean", "value_std_std"]:
                joined_df[col] = pd.to_numeric(joined_df[col], errors='coerce')

                top10_mean = top10[col].mean()
                bottom10_mean = bottom10[col].mean()
                print(f"For feature {featname}, {col}: top10 mean = {top10_mean:.3f}, bottom10 mean = {bottom10_mean:.3f}")
            # Calculate top20 and bottom20 subjects based on accuracy_avg.
            top20 = joined_df.nlargest(20, 'accuracy_avg')
            bottom20 = joined_df.nsmallest(20, 'accuracy_avg')

            # For each selected column, perform Mann-Whitney U test
            for col in ["value_mean_mean", "value_mean_std", "value_std_mean", "value_std_std"]:
                # Drop NA values before testing
                top_data = top20[col].dropna()
                bottom_data = bottom20[col].dropna()
                U, p_val = stats.mannwhitneyu(top_data, bottom_data, alternative='two-sided')
                # print(f"For feature {featname}, {col}: Mann-Whitney U = {U:.3f}, p-value = {p_val:.3f}")
        elif feats2level[featname] == '3d':
            continue
    

Processing folder: /data/storage1t/Turntaking/allnew_merged_wavs-features
Found 3083 npy files
0 files with all 0 values
For feature jitter, value_mean: Mann-Whitney U = 183.000, p-value = 0.655
For feature jitter, value_std: Mann-Whitney U = 185.000, p-value = 0.695
Processing folder: /data/storage1t/Turntaking/allnew_merged_wavs-features
Found 3083 npy files
0 files with all 0 values
For feature shimmer, value_mean: Mann-Whitney U = 198.000, p-value = 0.968
For feature shimmer, value_std: Mann-Whitney U = 197.000, p-value = 0.946
Processing folder: /data/storage1t/Turntaking/allnew_merged_wavs-features
Found 3083 npy files
All values larger than 500.0 in /data/storage1t/Turntaking/allnew_merged_wavs-features/f0/subj-2218_merged_5.wav_f0.npy
All nan values in /data/storage1t/Turntaking/allnew_merged_wavs-features/f0/subj-1105_merged_2.wav_f0.npy
All values larger than 500.0 in /data/storage1t/Turntaking/allnew_merged_wavs-features/f0/subj-1105_merged_14.wav_f0.npy
All values larger th